# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sys
import pandas as pd
import pickle

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download(['punkt', 'stopwords', 'wordnet', 'averaged_perceptron_tagger'])

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jwang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jwang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jwang\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\jwang\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
df = pd.read_sql_table('DisasterResponseTable', 'sqlite:///DisasterResponse.db')
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = df.message
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [4]:
# def tokenize(text):
    
#     text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) #normalize text   
#     stop_words = stopwords.words("english") # stopword list
#     words = word_tokenize(text) #tokenize
#     stemmed = [PorterStemmer().stem(w) for w in words] #stemming
#     #lemmatizing
#     lemmatized_tokens = [WordNetLemmatizer().lemmatize(w) for w in 
#                     stemmed if w not in stop_words] 
   
#     return lemmatized_tokens

def tokenize(text):
    '''
    INPUT 
        text -- Original text   
    OUTPUT
        Returns a processed text variable that was tokenized, lower cased,  stripped, and lemmatized
    '''
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
      ('vect', CountVectorizer(tokenizer=tokenize)),
      ('tfidf', TfidfTransformer()),
      ('clf', MultiOutputClassifier(RandomForestClassifier()))
      ]) 

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
#Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 45)
# train classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
def model_performance(model, X_test, y_test):
    category_names = list(df.columns[4:])
    y_pred = model.predict(X_test)

    for i in range(len(category_names)):
        print("------------------------------------------------------\n",
              "Category: {}".format(category_names[i]),
              "\n", 
              classification_report(y_test.iloc[:, i].values, y_pred[:, i]))

In [26]:
model_performance(pipeline, X_test, y_test)

------------------------------------------------------
 Category: related 
               precision    recall  f1-score   support

           0       0.64      0.47      0.54      1782
           1       0.85      0.92      0.89      6027

    accuracy                           0.82      7809
   macro avg       0.75      0.70      0.71      7809
weighted avg       0.80      0.82      0.81      7809

------------------------------------------------------
 Category: request 
               precision    recall  f1-score   support

           0       0.89      0.98      0.93      6479
           1       0.79      0.42      0.54      1330

    accuracy                           0.88      7809
   macro avg       0.84      0.70      0.74      7809
weighted avg       0.87      0.88      0.87      7809

------------------------------------------------------
 Category: offer 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      7771
           

 Category: aid_centers 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      7721
           1       0.00      0.00      0.00        88

    accuracy                           0.99      7809
   macro avg       0.49      0.50      0.50      7809
weighted avg       0.98      0.99      0.98      7809

------------------------------------------------------
 Category: other_infrastructure 
               precision    recall  f1-score   support

           0       0.95      1.00      0.98      7453
           1       0.23      0.01      0.02       356

    accuracy                           0.95      7809
   macro avg       0.59      0.50      0.50      7809
weighted avg       0.92      0.95      0.93      7809

------------------------------------------------------
 Category: weather_related 
               precision    recall  f1-score   support

           0       0.86      0.96      0.91      5566
           1       0.86      0.62      

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
parameters = {'clf__estimator__max_depth': [10, 50],
              'clf__estimator__min_samples_leaf':[2, 5]}

cv = GridSearchCV(pipeline, parameters)

In [10]:
cv

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                       

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
cv.fit(X_train, y_train)
model_performance(cv, X_test, y_test)

------------------------------------------------------

Category: related
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1782
           1       0.77      1.00      0.87      6027

    accuracy                           0.77      7809
   macro avg       0.39      0.50      0.44      7809
weighted avg       0.60      0.77      0.67      7809

------------------------------------------------------

Category: request
              precision    recall  f1-score   support

           0       0.83      1.00      0.91      6479
           1       0.00      0.00      0.00      1330

    accuracy                           0.83      7809
   macro avg       0.41      0.50      0.45      7809
weighted avg       0.69      0.83      0.75      7809

------------------------------------------------------

Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7771
           1     


------------------------------------------------------

Category: storm
              precision    recall  f1-score   support

           0       0.91      1.00      0.95      7068
           1       1.00      0.00      0.01       741

    accuracy                           0.91      7809
   macro avg       0.95      0.50      0.48      7809
weighted avg       0.91      0.91      0.86      7809

------------------------------------------------------

Category: fire
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7723
           1       0.00      0.00      0.00        86

    accuracy                           0.99      7809
   macro avg       0.49      0.50      0.50      7809
weighted avg       0.98      0.99      0.98      7809

------------------------------------------------------

Category: earthquake
              precision    recall  f1-score   support

           0       0.91      1.00      0.95      7069
           1    

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [12]:
pipeline2 = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ]))
    ])),

    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])


pipeline2.fit(X_train, y_train)
model_performance(pipeline2, X_test, y_test)

------------------------------------------------------

Category: related
              precision    recall  f1-score   support

           0       0.57      0.36      0.44      1782
           1       0.83      0.92      0.87      6027

    accuracy                           0.79      7809
   macro avg       0.70      0.64      0.66      7809
weighted avg       0.77      0.79      0.77      7809

------------------------------------------------------

Category: request
              precision    recall  f1-score   support

           0       0.89      0.97      0.93      6479
           1       0.72      0.40      0.52      1330

    accuracy                           0.87      7809
   macro avg       0.80      0.68      0.72      7809
weighted avg       0.86      0.87      0.86      7809

------------------------------------------------------

Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7771
           1     


------------------------------------------------------

Category: hospitals
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7722
           1       0.00      0.00      0.00        87

    accuracy                           0.99      7809
   macro avg       0.49      0.50      0.50      7809
weighted avg       0.98      0.99      0.98      7809

------------------------------------------------------

Category: shops
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7779
           1       0.00      0.00      0.00        30

    accuracy                           1.00      7809
   macro avg       0.50      0.50      0.50      7809
weighted avg       0.99      1.00      0.99      7809

------------------------------------------------------

Category: aid_centers
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7721
          

In [13]:
cv2 = GridSearchCV(pipeline2, param_grid=parameters)
cv2

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('features',
                                        FeatureUnion(n_jobs=None,
                                                     transformer_list=[('text_pipeline',
                                                                        Pipeline(memory=None,
                                                                                 steps=[('vect',
                                                                                         CountVectorizer(analyzer='word',
                                                                                                         binary=False,
                                                                                                         decode_error='strict',
                                                                                                         dtype=<class 'numpy.int64'

### 9. Export your model as a pickle file

In [14]:
# save the model to disk
model_filepath = 'classifier.pkl'
pickle.dump(cv2, open(model_filepath, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.